## Wykład 2 - Jak działa GPT
Wprowadzenie do biblioteki PyTorch i operacji wykorzystywanych w algorytmach klasyfikacji (softmax, entropia krzyżowa)

PyTorch zapewnia:
* obliczenia numeryczne na wielowymiarowych macierzach (tensorach)
* Automatyczne wyznaczanie gradientów (potrzebne do optymalizacji gradientowej)
* Wyskopoziomowe moduły do budowy sieci neuronowych
* Zapewnia większą elastyczność niż Keras i jest dużo przyjemniejszy w obsłudze niż Tensorflow 

In [1]:
import torch

### Macierze PyTorch

In [2]:
torch.zeros(4, 4)

tensor([[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]])

In [3]:
A = torch.arange(1, 10)
A

tensor([1, 2, 3, 4, 5, 6, 7, 8, 9])

In [4]:
# shape - sprawdza kształt tensora
A.shape

torch.Size([9])

In [8]:
# reshape zmienia kształt
A = A.reshape(3, 3)
A

tensor([[1, 2, 3],
        [4, 5, 6],
        [7, 8, 9]])

In [6]:
A.shape

torch.Size([3, 3])

In [9]:
# podstawienie wartości
A[0, 0] = 100

In [10]:
A

tensor([[100,   2,   3],
        [  4,   5,   6],
        [  7,   8,   9]])

In [11]:
A.sum()

tensor(144)

In [12]:
A.sum(axis=0)

tensor([111,  15,  18])

In [15]:
A.sum(axis=1)

tensor([105,  15,  24])

In [36]:
# keepdims=True zachowuje liczbę wymiarów tensora po operacji sumowania
A.sum(axis=1, keepdims=True)

tensor([[105],
        [ 15],
        [ 24]])

In [19]:
A + 3

tensor([[103,   5,   6],
        [  7,   8,   9],
        [ 10,  11,  12]])

In [20]:
A * 3

tensor([[300,   6,   9],
        [ 12,  15,  18],
        [ 21,  24,  27]])

In [23]:
# mnożenie macierzy
A @ A

tensor([[10029,   234,   339],
        [  462,    81,    96],
        [  795,   126,   150]])

#### Konkatenacja macierzy

In [24]:
a = torch.arange(1, 5)
b = torch.arange(1, 5)
print(a)
print(b)

tensor([1, 2, 3, 4])
tensor([1, 2, 3, 4])


In [25]:
torch.stack([a, b])

tensor([[1, 2, 3, 4],
        [1, 2, 3, 4]])

#### Losowanie

In [27]:
torch.randint(low=50, high=100, size=(2, 2))

tensor([[93, 80],
        [98, 61]])

In [28]:
torch.normal(mean=0, std=0.01, size=(2, 2))

tensor([[-0.0307,  0.0028],
        [ 0.0038, -0.0108]])

#### Logity
- dla sieci neuronowej klasyfikującej $n$ kategorii mamy w warstwie wyjściowej $n$ neuronów (np. klasyfikacja 200k tokenów)
- **Logity** to surowe wartości przed obliczeniem funkcji aktywacji
- Rozważmy przykład:
    - dwóch przykładów (wejść sieci) o czterech cechach ($x$)
    - wektora wag $W$ o wymiarze 4 x 3
    - klasyfikacji do 3 klas (czyli 3 neurony wyjściowe) 

In [30]:
x = torch.normal(mean=0, std=1, size=(2, 4))
x

tensor([[-1.8557, -0.7003, -0.0732, -1.6119],
        [ 0.5489,  1.8417, -0.2932, -0.3619]])

In [31]:
W = torch.normal(mean=0, std=1, size=(4, 3))
W

tensor([[ 0.7362, -0.6520,  1.4603],
        [ 0.4427, -1.3592,  1.8464],
        [ 1.2158, -1.1247,  0.3897],
        [-1.3426, -0.4803,  0.7843]])

In [32]:
logits = x @ W
print(logits.shape)
print(logits)

torch.Size([2, 3])
tensor([[ 0.3990,  3.0183, -5.2956],
        [ 1.3489, -2.3576,  3.8040]])


#### Softmax
$$softmax(z_i) = \frac{e^{z_i}}{\sum_{j=1}^{n} e^{z_j}}$$


In [33]:
exp_logits = torch.exp(logits)
exp_logits

tensor([[1.4903e+00, 2.0457e+01, 5.0136e-03],
        [3.8530e+00, 9.4650e-02, 4.4880e+01]])

In [34]:
exp_logits.sum(axis=1, keepdims=True)

tensor([[21.9528],
        [48.8276]])

In [37]:
# normalizujemy każdy wiersz, żeby sumował się do jedynki
# wyjścia po funkcji aktywacji softmax mają interpretację prawdopodobieństwa
probs = exp_logits/exp_logits.sum(axis=1, keepdims=True)
probs

tensor([[6.7886e-02, 9.3189e-01, 2.2838e-04],
        [7.8911e-02, 1.9384e-03, 9.1915e-01]])

In [38]:
# po co jest keepdims? - wymiary muszą się zgadzać
probs = exp_logits/exp_logits.sum(axis=1)
probs

RuntimeError: The size of tensor a (3) must match the size of tensor b (2) at non-singleton dimension 1

#### One hot encoding

In [39]:
import torch.nn.functional as F

In [41]:
# pierwszy argument - jaką liczbę zakodować
# drugi argument - ile jest klas (num_classes)
# kodowanie umieszcza jedynkę na pozycji odpowiadającej indeksowi pierwszego argumentu, w wektorze o długości n_classes
F.one_hot(torch.tensor(1), num_classes=5)

tensor([0, 1, 0, 0, 0])

In [42]:
# prawidłowe wyjścia sieci (etykiety) zakodowane jako one hot encoding
# dla pierwszego przykłądu prawidłową etykietą jest 0, a dla drugiego 2
y = F.one_hot(torch.tensor([0, 2]), num_classes=3)
y

tensor([[1, 0, 0],
        [0, 0, 1]])

#### Entropia krzyżowa
(dla pojedynczego przykładu)
$$ loss = -\sum_{i=1}^{n} y_i \log(\hat{y}_i)$$ 

In [43]:
probs

tensor([[6.7886e-02, 9.3189e-01, 2.2838e-04],
        [7.8911e-02, 1.9384e-03, 9.1915e-01]])

In [44]:
torch.log(probs)

tensor([[-2.6899, -0.0705, -8.3845],
        [-2.5394, -6.2459, -0.0843]])

In [46]:
y

tensor([[1, 0, 0],
        [0, 0, 1]])

In [45]:
# mnożenie (element po elemencie) wybiera logproby dla prawidłowych etykiet
y * torch.log(probs)

tensor([[-2.6899, -0.0000, -0.0000],
        [-0.0000, -0.0000, -0.0843]])

In [47]:
# sumowanie wszystkich przykładów - uwaga nawias jest potrzebny
(y * torch.log(probs)).sum()

tensor(-2.7742)

In [48]:
# pełna entropia krzyżowa - dodajemy minus i dzielimy przez liczbę przykładów, aby mieć średni koszt dla przykładu
- (y * torch.log(probs)).sum()/2

tensor(1.3871)

#### Gradienty
Źródło: [dokumentacja PyTorch](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html)

$$ Q = 3a^3 - b^2$$
$$\frac{\partial Q}{\partial a} = 9a^2$$
$$\frac{\partial Q}{\partial b} = -2b$$

In [49]:
a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)
Q = (3*a**3 - b**2).sum()
print(Q)

tensor(53., grad_fn=<SumBackward0>)


In [50]:
# wywołanie funkcji backward wylicza gradienty i zapisuje je w atrybucie grad odpowiednich tensorów
Q.backward()

In [51]:
a.grad

tensor([36., 81.])

In [52]:
b.grad

tensor([-12.,  -8.])

#### Prosta pętla optymalizacji gradientowej
Chcemy znaleźć minimum funkcji:
$$f(x)=(x−3)^2$$

In [55]:
x = torch.tensor([0.0], requires_grad=True)  # zaczynamy z x = 0

learning_rate = 0.1 # współczynnik uczenia
epochs = 30  # liczba iteracji

for epoch in range(epochs):
    loss = (x - 3) ** 2  # wartość funkcji kosztu
    
    loss.backward()  # wyznaczamy gradient
    
    with torch.no_grad():  # aktualizujemy wartość x - nie chcemy, żeby ta operacja była uwzględniana w wyliczaniu gradientów
        x -= learning_rate * x.grad
        x.grad.zero_()  # Zerujemy gradienty przed kolejną pętlą - to ważne!

    print(f"Krok {epoch+1}: x = {x.item()}, Loss = {loss.item()}")

print(f"Końcowa wartość x: {x.item()}")


Krok 1: x = 0.6000000238418579, Loss = 9.0
Krok 2: x = 1.0800000429153442, Loss = 5.760000228881836
Krok 3: x = 1.4639999866485596, Loss = 3.6863999366760254
Krok 4: x = 1.7711999416351318, Loss = 2.3592960834503174
Krok 5: x = 2.0169599056243896, Loss = 1.5099495649337769
Krok 6: x = 2.2135679721832275, Loss = 0.9663678407669067
Krok 7: x = 2.370854377746582, Loss = 0.6184753179550171
Krok 8: x = 2.4966835975646973, Loss = 0.39582422375679016
Krok 9: x = 2.597346782684326, Loss = 0.2533273994922638
Krok 10: x = 2.677877426147461, Loss = 0.16212961077690125
Krok 11: x = 2.7423019409179688, Loss = 0.10376295447349548
Krok 12: x = 2.793841600418091, Loss = 0.06640829145908356
Krok 13: x = 2.835073232650757, Loss = 0.042501285672187805
Krok 14: x = 2.868058681488037, Loss = 0.027200838550925255
Krok 15: x = 2.894446849822998, Loss = 0.01740851067006588
Krok 16: x = 2.915557384490967, Loss = 0.011141467839479446
Krok 17: x = 2.932446002960205, Loss = 0.007130555342882872
Krok 18: x = 2.945

In [56]:
x

tensor([2.9963], requires_grad=True)

In [57]:
# metoda item wyjmuje liczbę z tensora (tylko dla jednowymiarowych)
x.item()

2.996286153793335

#### Dodatki - elementy Pythona występujące na lab 2 (za tydzień)
* (oprócz list) nie trzeba będzie tego implementować, ale dobrze jest rozumieć kod

##### Listy

In [58]:
# listy
lista = ['a', 'b', 'c']
print(lista)

['a', 'b', 'c']


In [60]:
# ostatni element listy
lista[-1]

'c'

In [61]:
# elementy od elementu o indeksie 1 do końca listy
lista[1:]

['b', 'c']

In [62]:
for el in lista:
    print(el)

a
b
c


##### Zip
* łączy elementy iterowalne (np. listy) tworząc pary (lub krotki) z różnych list
* Kończy jak krótszy z elementów iterowalnych się wyczerpie

In [63]:
lista1 = ['a', 'b', 'c']
lista2 = ['d', 'e', 'f']

In [64]:
print(list(zip(lista1, lista2)))

[('a', 'd'), ('b', 'e'), ('c', 'f')]


In [65]:
for ch1, ch2 in zip(lista1, lista2):
    print(ch1, ch2)

a d
b e
c f


In [67]:
lista1

['a', 'b', 'c']

In [68]:
lista2

['d', 'e', 'f']

In [66]:
# teraz bierzemy z drugiej listy od elementu o indeksie 1 i mamy tylko dwie pary
for ch1, ch2 in zip(lista1, lista2[1:]):
    print(ch1, ch2)

a e
b f


##### List Comprehension

In [ ]:
[i for i in range(10)]

In [ ]:
[i**2 for i in range(10)]

##### NumPy
* NumPy jest podstawową biblioteką do obliczeń numerycznych w Pythonie
* Większość składni i funkcjonalności jest bardzo podobna jak dla tensorów PyTorcha
* Będziemy czasem korzystać

In [ ]:
import numpy as np

In [ ]:
# tego losowania niestety PyTorch nie zapewnia w wygodny sposób więc wykorzystamy bibliotekę numpy
# losujemy liczby z prawdopodobieństwami podanymi w p
np.random.choice(3, p=[0.1, 0.2, 0.7])

In [ ]:
# losowanie liczb całkowitych z rozkładu jednostajnego
np.random.randint(0, 10, size=5)